# Dataframe Fundamentals 

### Dataframe
In Apache Spark, a DataFrame is a distributed collection of rows under named columns. It is conceptually equivalent to a table in a relational database, an Excel sheet with Column headers, or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. It also shares some common characteristics with RDD:

* Immutable in nature : We can create DataFrame / RDD once but can’t change it. And we can transform a DataFrame / RDD after applying transformations.
* Lazy Evaluations: Which means that a task is not executed until an action is performed.
* Distributed: RDD and DataFrame both are distributed in nature.

In [1]:
import findspark
findspark.init()
 
import pyspark
from pyspark import SparkContext as sc

### Create a `SparkSession` app object

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark1 = SparkSession.builder.appName('Basics').getOrCreate()

### Read in a JSON file and examine the data

In [4]:
df = spark1.read.json('Data/people.json')

#### Unlike Pandas DataFrame, it does not show itself when called

In [5]:
df

DataFrame[age: bigint, name: string]

#### You have to call show() method to evaluate it i.e. show it

In [6]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  23|  Bruce|
|  99| Donald|
|  14|   Jane|
|  41|   Jack|
|  22|   Jill|
|  35|   John|
+----+-------+



### The data schema
Use `printSchema()` to show he schema of the data. Note, how tightly it is integrated to the SQL-like framework. You can even see that the schema accepts `null` values because nullable property is set `True`.

In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



#### Fortunately a simple `columns` method exists to get column names back as a Python list

In [8]:
df.columns

['age', 'name']

#### Similar to Pandas, the `describe` method is used for the statistical summary. But unlike Pandas, calling only `describe()` returns a DataFrame! This is due to lazy evaluation - the actual computation is delayed as much as possible.

In [9]:
df.describe()

DataFrame[summary: string, age: string, name: string]

#### We have to call `show` again

In [10]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 8|      9|
|   mean|            35.375|   null|
| stddev|27.165826747167902|   null|
|    min|                14|   Andy|
|    max|                99|Michael|
+-------+------------------+-------+



#### There is `summary` method for more stats

In [ ]:
df.summary().show()

### How you can define your own Data Schema
#### Import data types and structure types to build the data schema yourself

In [ ]:
from pyspark.sql.types import StructField, IntegerType, StringType, StructType

#### Define your data schema by supplying name and data types to the structure fields you will be importing

In [ ]:
data_schema = [StructField('age',IntegerType(),True),
StructField('name',StringType(),True)]

#### Now create a `StrucType` with this schema as field

In [ ]:
final_struc = StructType(fields=data_schema)

#### Now read in the same old JSON with this new schema

In [ ]:
df = spark1.read.json('Data/people.json',schema=final_struc)
df.show()

## Read Files CSV and JSON

In [11]:

peopleDF = spark1.read.json("Data/people.json")


In [13]:
# DataFrames can be saved as Parquet files, maintaining the schema information.
peopleDF.write.parquet("people.parquet")
 


In [14]:
!ls *.parquet

_SUCCESS
part-00000-8a7b440a-82d0-4871-bd78-6f402f475b58-c000.snappy.parquet


In [17]:
# The result of loading a parquet file is also a DataFrame.
parquetFileDF = spark1.read.parquet("people.parquet")


In [18]:
type(parquetFileDF)

pyspark.sql.dataframe.DataFrame

In [19]:
#
teenagers_DF = parquetFileDF.filter("age >= 13 AND age <= 19").select(['name','age'])
teenagers_DF.show()

+------+---+
|  name|age|
+------+---+
|Justin| 19|
|  Jane| 14|
+------+---+



In [20]:
# Parquet files can also be used to create a temporary view and then used in SQL statements.

parquetFileDF.createOrReplaceTempView("parquetFileSQL")
teenagers_SQL = spark1.sql("SELECT name,age FROM parquetFileSQL WHERE age >= 13 AND age <= 19")
teenagers_SQL.show()
 

+------+---+
|  name|age|
+------+---+
|Justin| 19|
|  Jane| 14|
+------+---+



In [21]:
salesinfo = spark1\
            .read\
            .option("inferSchema", "true")\
            .option("header", "true")\
            .csv("./Data/sales_info.csv")




In [22]:
salesinfo.write.parquet("sales.parquet")

In [23]:
salesFile = spark1.read.parquet("sales.parquet")
salesFile.createOrReplaceTempView("SalesParquet")
All_Sales = spark1.sql("SELECT * FROM SalesParquet")
All_Sales.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+

